In [19]:
# ИМПОРТ СОВРЕМЕННЫХ БИБЛИОТЕК
import duckdb
import polars as pl
import jax.numpy as jnp
import jax
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import json
from datetime import datetime
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

print("✅ Библиотеки загружены")
print(f" DuckDB: {duckdb.__version__}")
print(f" Polars: {pl.__version__}")

✅ Библиотеки загружены
 DuckDB: 1.4.3
 Polars: 1.37.0


In [39]:

print("🎲 ГЕНЕРАЦИЯ ТЕСТОВЫХ ДАННЫХ (простая версия)")

n_samples = 100_000
np.random.seed(42)

# Базовые категории и их веса
categories = {
    'работа': 0.15,
    'игры': 0.20, 
    'развлечения': 0.15,
    'новости': 0.10,
    'техника': 0.08,
    'животные': 0.07,
    'еда': 0.06,
    'спорт': 0.05,
    'образование': 0.07,
    'путешествия': 0.07
}

# Генерация данных
data = []
base_time = int(datetime(2023, 10, 1).timestamp())

for i in range(n_samples):
    # Платформа (50/50 распределение)
    platform = 'desktop' if np.random.random() < 0.5 else 'touch'
    
    # Время с паттернами
    hour = i % 24
    
    # Выбор категории с учетом платформы и времени
    if platform == 'desktop' and 9 <= hour <= 17:
        # Desktop в рабочее время - больше рабочих запросов
        cat_weights = list(categories.values())
        cat_weights[0] *= 2  # увеличиваем вес "работы"
        cat_weights[1] *= 0.5  # уменьшаем вес "игр"
        cat_weights = [w/sum(cat_weights) for w in cat_weights]
    elif platform == 'touch' and 18 <= hour <= 23:
        # Mobile вечером - больше развлечений
        cat_weights = list(categories.values())
        cat_weights[1] *= 1.5  # увеличиваем вес "игр"
        cat_weights[2] *= 1.5  # увеличиваем вес "развлечений"
        cat_weights[0] *= 0.3  # уменьшаем вес "работы"
        cat_weights = [w/sum(cat_weights) for w in cat_weights]
    else:
        cat_weights = list(categories.values())
    
    # Выбираем категорию
    category = np.random.choice(list(categories.keys()), p=cat_weights)
    
    # Создаем запрос на основе категории
    variations = {
        'работа': [f'работа удаленно {i%100}', f'офис программы {i%50}', 
                   f'проект менеджмент {i%20}', 'дедлайн срочно'],
        'игры': [f'игры мобильные {i%100}', f'гейминг {i%50}', 
                 f'топ игр {2023-i%5}', 'киберспорт трансляции'],
        'развлечения': [f'фильмы {i%100}', f'сериалы онлайн {i%50}',
                       f'музыка {2023-i%3}', 'ютуб каналы топ'],
        'новости': [f'новости {i%100}', 'политика сегодня', 
                   'экономика новости', 'технологии события'],
        'техника': [f'телефон новый {i%100}', f'ноутбук {i%50}',
                   'гаджеты обзор', 'техника характеристики'],
        'животные': [f'котики {i%100}', 'милые животные видео',
                    'питомцы уход', 'собаки породы'],
        'еда': [f'рецепты {i%100}', 'быстрая еда рецепты',
               'здоровое питание', 'кулинария советы'],
        'спорт': [f'футбол {i%100}', 'спорт новости сегодня',
                 'тренировки дома', 'спорт трансляции'],
        'образование': [f'курсы онлайн {i%100}', 'программирование обучение',
                       'языки изучение', 'навыки работы'],
        'путешествия': [f'путешествия {i%100}', 'билеты дешевые',
                       'отели бронирование', 'туризм советы']
    }
    
    query = np.random.choice(variations[category])
    
    # Timestamp
    ts = base_time + (i // 1000) * 3600 + hour * 3600 + np.random.randint(0, 3600)
    
    data.append({
        'query': query,
        'platform': platform,
        'category': category,
        'ts': ts,
        'hour': hour
    })

# Создаем Polars DataFrame
df_polars = pl.DataFrame(data)

print(f"✅ Сгенерировано {df_polars.shape[0]:,} записей")
print("\n📊 СТАТИСТИКА ДАННЫХ:")
print(df_polars.describe())

# Быстрая визуализация распределения
platform_dist = df_polars.group_by('platform').agg(pl.count()).sort('platform')
fig = px.pie(platform_dist.to_pandas(), values='count', names='platform',
             title='Распределение запросов по платформам')
fig.show()

🎲 ГЕНЕРАЦИЯ ТЕСТОВЫХ ДАННЫХ (простая версия)
✅ Сгенерировано 100,000 записей

📊 СТАТИСТИКА ДАННЫХ:
shape: (9, 6)
┌────────────┬────────────────┬──────────┬──────────┬──────────────┬──────────┐
│ statistic  ┆ query          ┆ platform ┆ category ┆ ts           ┆ hour     │
│ ---        ┆ ---            ┆ ---      ┆ ---      ┆ ---          ┆ ---      │
│ str        ┆ str            ┆ str      ┆ str      ┆ f64          ┆ f64      │
╞════════════╪════════════════╪══════════╪══════════╪══════════════╪══════════╡
│ count      ┆ 100000         ┆ 100000   ┆ 100000   ┆ 100000.0     ┆ 100000.0 │
│ null_count ┆ 0              ┆ 0        ┆ 0        ┆ 0.0          ┆ 0.0      │
│ mean       ┆ null           ┆ null     ┆ null     ┆ 1.6963e9     ┆ 11.49936 │
│ std        ┆ null           ┆ null     ┆ null     ┆ 106867.18186 ┆ 6.922098 │
│ min        ┆ билеты дешевые ┆ desktop  ┆ еда      ┆ 1.6961e9     ┆ 0.0      │
│ 25%        ┆ null           ┆ null     ┆ null     ┆ 1.6962e9     ┆ 5.0      │
│ 50%  

/tmp/ipykernel_90024/1732144886.py:96: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)



In [40]:
df_polars.head()

query,platform,category,ts,hour
str,str,str,i64,i64
"""отели бронирование""","""desktop""","""путешествия""",1696108695,0
"""топ игр 2022""","""touch""","""игры""",1696112438,1
"""навыки работы""","""desktop""","""образование""",1696117719,2
"""офис программы 3""","""touch""","""работа""",1696120791,3
"""киберспорт трансляции""","""touch""","""игры""",1696122955,4


In [33]:
# АНАЛИЗ ДАННЫХ С POLARS
print("\n📊 АНАЛИЗ С POLARS (ВЫСОКОПРОИЗВОДИТЕЛЬНЫЙ)")

# Добавляем время и категории
df_polars = df_polars.with_columns([
    pl.from_epoch('ts').alias('datetime'),
    (pl.col('hour') >= 9) & (pl.col('hour') <= 17).alias('is_work_hours'),
    pl.col('hour').is_between(18, 23).alias('is_evening')
])

# Категоризация запросов
def categorize_query_polars(query: str) -> str:
    query_lower = query.lower()
    if 'работа' in query_lower or 'офис' in query_lower or 'дедлайн' in query_lower:
        return 'работа'
    elif 'игр' in query_lower or 'гейм' in query_lower:
        return 'игры'
    elif 'фильм' in query_lower or 'сериал' in query_lower or 'музык' in query_lower:
        return 'развлечения'
    elif 'новост' in query_lower:
        return 'новости'
    elif 'телефон' in query_lower or 'ноутбук' in query_lower or 'гаджет' in query_lower:
        return 'техника'
    elif 'котик' in query_lower or 'животн' in query_lower or 'собак' in query_lower:
        return 'животные'
    elif 'рецепт' in query_lower or 'еда' in query_lower:
        return 'еда'
    else:
        return 'другое'

# Векторизованная категоризация
df_polars = df_polars.with_columns([
    pl.col('query').map_elements(categorize_query_polars).alias('category')
])

print("📈 ОСНОВНАЯ СТАТИСТИКА:")
print(df_polars.describe())



📊 АНАЛИЗ С POLARS (ВЫСОКОПРОИЗВОДИТЕЛЬНЫЙ)
📈 ОСНОВНАЯ СТАТИСТИКА:
shape: (9, 8)
┌────────────┬─────────────┬──────────┬──────────┬────────────┬──────────┬────────────┬────────────┐
│ statistic  ┆ query       ┆ platform ┆ category ┆ ts         ┆ hour     ┆ datetime   ┆ is_evening │
│ ---        ┆ ---         ┆ ---      ┆ ---      ┆ ---        ┆ ---      ┆ ---        ┆ ---        │
│ str        ┆ str         ┆ str      ┆ str      ┆ f64        ┆ f64      ┆ str        ┆ f64        │
╞════════════╪═════════════╪══════════╪══════════╪════════════╪══════════╪════════════╪════════════╡
│ count      ┆ 100000      ┆ 100000   ┆ 100000   ┆ 100000.0   ┆ 100000.0 ┆ 100000     ┆ 100000.0   │
│ null_count ┆ 0           ┆ 0        ┆ 0        ┆ 0.0        ┆ 0.0      ┆ 0          ┆ 0.0        │
│ mean       ┆ null        ┆ null     ┆ null     ┆ 1.6963e9   ┆ 0.37501  ┆ 2023-10-03 ┆ 0.24996    │
│            ┆             ┆          ┆          ┆            ┆          ┆ 10:29:55.9 ┆            │
│         

In [34]:

# Быстрая агрегация с Polars
platform_stats = df_polars.group_by('platform').agg([
    pl.len().alias('total_queries'),
    pl.col('category').n_unique().alias('unique_categories'),
    pl.col('hour').mean().alias('avg_hour'),
    pl.col('hour').std().alias('std_hour')
]).sort('platform')

print("\n📱 СТАТИСТИКА ПО ПЛАТФОРМАМ:")
print(platform_stats)


📱 СТАТИСТИКА ПО ПЛАТФОРМАМ:
shape: (2, 5)
┌──────────┬───────────────┬───────────────────┬──────────┬──────────┐
│ platform ┆ total_queries ┆ unique_categories ┆ avg_hour ┆ std_hour │
│ ---      ┆ ---           ┆ ---               ┆ ---      ┆ ---      │
│ str      ┆ u32           ┆ u32               ┆ f64      ┆ f64      │
╞══════════╪═══════════════╪═══════════════════╪══════════╪══════════╡
│ desktop  ┆ 50009         ┆ 8                 ┆ 0.375872 ┆ 0.484352 │
│ touch    ┆ 49991         ┆ 8                 ┆ 0.374147 ┆ 0.483907 │
└──────────┴───────────────┴───────────────────┴──────────┴──────────┘


In [5]:
# ИНТЕРАКТИВНЫЙ АНАЛИЗ С PLOTLY
print("\n📊 СОЗДАНИЕ ИНТЕРАКТИВНЫХ ГРАФИКОВ")

# 1. Распределение по платформам (интерактивная круговая диаграмма)
platform_counts = df_polars.group_by('platform').agg(pl.count().alias('count'))

fig1 = px.pie(platform_counts.to_pandas(), 
              values='count', 
              names='platform',
              title='Распределение запросов по платформам',
              color_discrete_sequence=px.colors.qualitative.Set3,
              hole=0.3)

fig1.update_traces(textposition='inside', textinfo='percent+label')
fig1.update_layout(showlegend=True)
fig1.show()



📊 СОЗДАНИЕ ИНТЕРАКТИВНЫХ ГРАФИКОВ


/tmp/ipykernel_90024/2902849337.py:5: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)



In [17]:

# 2. Топ запросов по платформам (интерактивная столбчатая диаграмма)
top_n = 15

# Получаем топ запросов для каждой платформы
top_desktop = (df_polars.filter(pl.col('platform') == 'desktop')
               .group_by('query')
               .agg(pl.len().alias('count'))
               .sort('count', descending=True)
               .head(top_n))

top_touch = (df_polars.filter(pl.col('platform') == 'touch')
             .group_by('query')
             .agg(pl.len().alias('count'))
             .sort('count', descending=True)
             .head(top_n))

# Создаем объединенный DataFrame для визуализации
top_combined = pl.concat([
    top_desktop.with_columns(pl.lit('desktop').alias('platform')),
    top_touch.with_columns(pl.lit('touch').alias('platform'))
]).to_pandas()

fig2 = px.bar(top_combined, 
              x='query', 
              y='count', 
              color='platform',
              barmode='group',
              title=f'Топ-{top_n} запросов по платформам',
              color_discrete_map={'desktop': '#636EFA', 'touch': '#EF553B'},
              labels={'query': 'Запрос', 'count': 'Количество'})

fig2.update_layout(xaxis_title='Запрос',
                   yaxis_title='Количество запросов',
                   xaxis_tickangle=-45,
                   height=500)
fig2.show()

# 3. Активность по часам (интерактивный линейный график)
hourly_activity = (df_polars.group_by(['platform', 'hour'])
                   .agg(pl.len().alias('count'))
                   .sort(['platform', 'hour'])
                   .to_pandas())

fig3 = px.line(hourly_activity, 
               x='hour', 
               y='count', 
               color='platform',
               title='Активность поиска по часам суток',
               markers=True,
               color_discrete_map={'desktop': '#636EFA', 'touch': '#EF553B'})

# Добавляем зоны для рабочего времени и вечера
fig3.add_vrect(x0=9, x1=17, 
               annotation_text="Рабочее время", 
               annotation_position="top left",
               fillcolor="green", opacity=0.1, line_width=0)

fig3.add_vrect(x0=18, x1=23, 
               annotation_text="Вечер", 
               annotation_position="top left",
               fillcolor="orange", opacity=0.1, line_width=0)

fig3.update_layout(xaxis_title='Час суток',
                   yaxis_title='Количество запросов',
                   xaxis=dict(tickmode='linear', dtick=2),
                   height=500)
fig3.show()

# 4. Heatmap активности (дни × часы × платформы)
# Создаем фиктивный день недели для визуализации
hourly_platform = (df_polars.group_by(['platform', 'hour'])
                   .agg(pl.len().alias('count'))
                   .pivot(index='hour', on='platform', values='count')
                   .fill_null(strategy = "mean")).to_pandas()

fig4 = go.Figure(data=go.Heatmap(
    z=hourly_platform.values,
    x=hourly_platform.columns,
    y=hourly_platform.index,
    colorscale='Viridis',
    colorbar=dict(title="Запросы")
))

fig4.update_layout(
    title='Heatmap активности по платформам и часам',
    xaxis_title='Платформа',
    yaxis_title='Час суток',
    height=400
)
fig4.show()

In [41]:
# СТАТИСТИЧЕСКИЙ АНАЛИЗ С DUCKDB (SQL-подобный синтаксис)
print("\n📈 СТАТИСТИЧЕСКИЙ АНАЛИЗ С DUCKDB")

# Создаем DuckDB соединение и загружаем данные
con = duckdb.connect()

# Регистрируем Polars DataFrame в DuckDB
con.register('search_data', df_polars)



📈 СТАТИСТИЧЕСКИЙ АНАЛИЗ С DUCKDB


In [42]:
df_polars.head()

query,platform,category,ts,hour
str,str,str,i64,i64
"""отели бронирование""","""desktop""","""путешествия""",1696108695,0
"""топ игр 2022""","""touch""","""игры""",1696112438,1
"""навыки работы""","""desktop""","""образование""",1696117719,2
"""офис программы 3""","""touch""","""работа""",1696120791,3
"""киберспорт трансляции""","""touch""","""игры""",1696122955,4


In [43]:
print("1️⃣ ОСНОВНЫЕ МЕТРИКИ:")

basic_stats_h = con.execute("""
    SELECT 
        platform,
        COUNT(*) as total_queries,
        COUNT(DISTINCT query) as unique_queries,
        AVG(hour) as avg_hour,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hour) as median_hour,
        MIN(hour) as min_hour,
        MAX(hour) as max_hour,
        STDDEV(hour) as std_hour
    FROM search_data
    GROUP BY platform
    ORDER BY platform
""").df()

print(basic_stats_h)

1️⃣ ОСНОВНЫЕ МЕТРИКИ:
  platform  total_queries  unique_queries   avg_hour  median_hour  min_hour  \
0  desktop          50009            1251  11.488572         11.0         0   
1    touch          49991            1251  11.510152         12.0         0   

   max_hour  std_hour  
0        23  6.920920  
1        23  6.923329  


In [45]:

# 1. Базовые статистики через SQL
print("1️⃣ ОСНОВНЫЕ МЕТРИКИ:")

basic_stats = con.execute("""
    SELECT 
        platform,
        COUNT(*) as total_queries,
        COUNT(DISTINCT query) as unique_queries,
        AVG(ts) as ts,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY ts) as median_hour,
        MIN(ts) as min_ts,
        MAX(ts) as max_ts,
        STDDEV(ts) as std_ts
    FROM search_data
    GROUP BY platform
    ORDER BY platform
""").df()

print(basic_stats)


1️⃣ ОСНОВНЫЕ МЕТРИКИ:
  platform  total_queries  unique_queries            ts   median_hour  \
0  desktop          50009            1251  1.696329e+09  1.696329e+09   
1    touch          49991            1251  1.696329e+09  1.696329e+09   

       min_ts      max_ts         std_ts  
0  1696107729  1696550343  106893.395733  
1  1696108018  1696550348  106841.710471  


In [46]:

# 2. Распределение по категориям
print("\n2️⃣ РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ:")

category_stats = con.execute("""
    WITH category_counts AS (
        SELECT 
            platform,
            category,
            COUNT(*) as count,
            COUNT(*) * 1.0 / SUM(COUNT(*)) OVER (PARTITION BY platform) as percentage
        FROM search_data
        GROUP BY platform, category
    )
    SELECT 
        platform,
        category,
        count,
        ROUND(percentage * 100, 2) as percentage
    FROM category_counts
    ORDER BY platform, count DESC
""").pl()


# Визуализация распределения категорий
fig5 = px.bar(category_stats, 
              x='category', 
              y='percentage',
              color='platform',
              barmode='group',
              title='Распределение запросов по категориям (%)',
              color_discrete_map={'desktop': '#636EFA', 'touch': '#EF553B'},
              labels={'percentage': 'Доля, %', 'category': 'Категория'})

fig5.update_layout(xaxis_title='Категория',
                   yaxis_title='Доля запросов (%)',
                   xaxis_tickangle=-45,
                   height=500)
fig5.show()



2️⃣ РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ:


In [47]:

# 3. Разница в распределении по часам
print("\n3️⃣ СРАВНЕНИЕ РАСПРЕДЕЛЕНИЙ ПО ЧАСАМ:")

hour_distribution = con.execute("""
    SELECT 
        hour,
        SUM(CASE WHEN platform = 'desktop' THEN 1 ELSE 0 END) as desktop_count,
        SUM(CASE WHEN platform = 'touch' THEN 1 ELSE 0 END) as touch_count,
        desktop_count * 1.0 / SUM(desktop_count) OVER () as desktop_pct,
        touch_count * 1.0 / SUM(touch_count) OVER () as touch_pct
    FROM search_data
    GROUP BY hour
    ORDER BY hour
""").df()

# Визуализация разницы
fig6 = go.Figure()

fig6.add_trace(go.Scatter(
    x=hour_distribution['hour'],
    y=hour_distribution['desktop_pct'] * 100,
    mode='lines+markers',
    name='Desktop',
    line=dict(color='#636EFA', width=3),
    marker=dict(size=8)
))

fig6.add_trace(go.Scatter(
    x=hour_distribution['hour'],
    y=hour_distribution['touch_pct'] * 100,
    mode='lines+markers',
    name='Touch',
    line=dict(color='#EF553B', width=3),
    marker=dict(size=8)
))

fig6.update_layout(
    title='Распределение запросов по часам (нормализованное)',
    xaxis_title='Час суток',
    yaxis_title='Доля запросов (%)',
    xaxis=dict(tickmode='linear', dtick=2),
    height=500,
    hovermode='x unified'
)

fig6.show()


3️⃣ СРАВНЕНИЕ РАСПРЕДЕЛЕНИЙ ПО ЧАСАМ:
